<a href="https://colab.research.google.com/github/BountyHunter1999/Machine-Learning-Zoom-Camp/blob/main/Homeworks/Homework%20%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Modules

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

# Getting The Data and First Glance at it

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
cols = ['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']

In [4]:
pd.read_csv("AB_NYC_2019.csv").shape # we don't need all of these

(48895, 16)

In [5]:
df = pd.read_csv("AB_NYC_2019.csv", usecols=cols) # getting only what we need

In [6]:
df.shape

(48895, 10)

## Looking At The Data

In [7]:
df.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,NaN,1,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


In [8]:
df.head().T

,0,1,2,3,4
neighbourhood_group,Brooklyn,Manhattan,Manhattan,Brooklyn,Manhattan
latitude,40.64749,40.75362,40.80902,40.68514,40.79851
longitude,-73.97237,-73.98377,-73.9419,-73.95976,-73.94399
room_type,Private room,Entire home/apt,Private room,Entire home/apt,Entire home/apt
price,149,225,150,89,80
minimum_nights,1,1,3,1,10
number_of_reviews,9,45,0,270,9
reviews_per_month,0.21,0.38,NaN,4.64,0.1
calculated_host_listings_count,6,2,1,1,1
availability_365,365,355,365,194,0


## Checking Null Values

In [9]:
df.isnull().sum()

neighbourhood_group                   0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [10]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [11]:
df.isnull().sum()

neighbourhood_group               0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

# DATA PREPARATION

### knowing about the data

In [12]:
df.dtypes

neighbourhood_group                object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [13]:
df.dtypes == "object"

neighbourhood_group                True
latitude                          False
longitude                         False
room_type                          True
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

In [14]:
df.dtypes[df.dtypes == "object"]

neighbourhood_group    object
room_type              object
dtype: object

In [15]:
df.dtypes[df.dtypes == 'object'].index

Index(['neighbourhood_group', 'room_type'], dtype='object')

In [16]:
categorical_cols = list(df.dtypes[df.dtypes == 'object'].index)
categorical_cols

['neighbourhood_group', 'room_type']

In [17]:
df.columns = df.columns.str.lower()

In [18]:
df[categorical_cols[0]][:4]

0     Brooklyn
1    Manhattan
2    Manhattan
3     Brooklyn
Name: neighbourhood_group, dtype: object

In [19]:
df[categorical_cols[0]][:4].str.lower().str.replace(" ", "_")

0     brooklyn
1    manhattan
2    manhattan
3     brooklyn
Name: neighbourhood_group, dtype: object

In [20]:
for c in categorical_cols:
  df[c] = df[c].str.lower().str.replace(" ", "_")

In [21]:
df.head().T

,0,1,2,3,4
neighbourhood_group,brooklyn,manhattan,manhattan,brooklyn,manhattan
latitude,40.64749,40.75362,40.80902,40.68514,40.79851
longitude,-73.97237,-73.98377,-73.9419,-73.95976,-73.94399
room_type,private_room,entire_home/apt,private_room,entire_home/apt,entire_home/apt
price,149,225,150,89,80
minimum_nights,1,1,3,1,10
number_of_reviews,9,45,0,270,9
reviews_per_month,0.21,0.38,0.0,4.64,0.1
calculated_host_listings_count,6,2,1,1,1
availability_365,365,355,365,194,0


In [22]:
# df["above_average"] = (df.price >= 152).astype(int)

In [23]:
# df[:3].T

# Question #1

In [24]:
df[categorical_cols].nunique()

neighbourhood_group    5
room_type              3
dtype: int64

In [25]:
df["neighbourhood_group"].value_counts()

manhattan        21661
brooklyn         20104
queens            5666
bronx             1091
staten_island      373
Name: neighbourhood_group, dtype: int64

In [26]:
train_test_split?


1.   First line split the dataset into (train + val) and test dataset 
  - (80%, 20%) with random seed of 42
2.   the second line we split the (train + val) into train and val
  - `.25` is there instead of `0.2` as we are splitting from the 80% of the original data
  - so 20% of 80% is `0.25`  





In [27]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [28]:
20/80

0.25

In [29]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [30]:
df_train.head().T

,13575,48476,44499,17382,14638
neighbourhood_group,brooklyn,manhattan,bronx,brooklyn,manhattan
latitude,40.7276,40.70847,40.83149,40.66448,40.74118
longitude,-73.94495,-74.00498,-73.92766,-73.99407,-74.00012
room_type,entire_home/apt,private_room,entire_home/apt,entire_home/apt,private_room
price,99,57,70,130,110
minimum_nights,3,1,40,2,1
number_of_reviews,29,0,0,3,48
reviews_per_month,0.7,0.0,0.0,0.08,1.8
calculated_host_listings_count,13,1,1,1,2
availability_365,50,7,0,0,67


In [31]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [32]:
df_full_train = df_full_train.reset_index(drop=True)
df_full_train[:4]

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,brooklyn,40.71577,-73.95530,entire_home/apt,295,3,11,0.87,1,1
1,manhattan,40.84917,-73.94048,private_room,70,2,2,0.16,1,0
2,brooklyn,40.68993,-73.95947,private_room,58,2,0,0.00,2,0
3,brooklyn,40.68427,-73.93118,entire_home/apt,75,3,87,4.91,1,267


In [33]:
df_train.head().T

,0,1,2,3,4
neighbourhood_group,brooklyn,manhattan,bronx,brooklyn,manhattan
latitude,40.7276,40.70847,40.83149,40.66448,40.74118
longitude,-73.94495,-74.00498,-73.92766,-73.99407,-74.00012
room_type,entire_home/apt,private_room,entire_home/apt,entire_home/apt,private_room
price,99,57,70,130,110
minimum_nights,3,1,40,2,1
number_of_reviews,29,0,0,3,48
reviews_per_month,0.7,0.0,0.0,0.08,1.8
calculated_host_listings_count,13,1,1,1,2
availability_365,50,7,0,0,67


In [34]:
y_train = df_train['price']
y_train.head()

0     99
1     57
2     70
3    130
4    110
Name: price, dtype: int64

In [35]:
y_val = df_val['price']
y_test = df_test['price']

In [36]:
# y_btrain = df_train['above_average']
# y_bval = df_val['above_average']
# y_btest = df_test['above_average']

In [37]:
del df_train['price']
del df_val['price']
del df_test['price']

In [38]:
# del df_train['above_average']
# del df_val['above_average']
# del df_test['above_average']

In [39]:
df_train.head().T

,0,1,2,3,4
neighbourhood_group,brooklyn,manhattan,bronx,brooklyn,manhattan
latitude,40.7276,40.70847,40.83149,40.66448,40.74118
longitude,-73.94495,-74.00498,-73.92766,-73.99407,-74.00012
room_type,entire_home/apt,private_room,entire_home/apt,entire_home/apt,private_room
minimum_nights,3,1,40,2,1
number_of_reviews,29,0,0,3,48
reviews_per_month,0.7,0.0,0.0,0.08,1.8
calculated_host_listings_count,13,1,1,1,2
availability_365,50,7,0,0,67


# Question #2

In [40]:
df_train.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [41]:
df.corr()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.084788,0.033939,0.024869,-0.015389,-0.018758,0.019517,-0.010983
longitude,0.084788,1.000000,-0.150019,-0.062747,0.059094,0.138516,-0.114713,0.082731
price,0.033939,-0.150019,1.000000,0.042799,-0.047954,-0.050564,0.057472,0.081829
minimum_nights,0.024869,-0.062747,0.042799,1.000000,-0.080116,-0.124905,0.127960,0.144303
number_of_reviews,-0.015389,0.059094,-0.047954,-0.080116,1.000000,0.589407,-0.072376,0.172028
reviews_per_month,-0.018758,0.138516,-0.050564,-0.124905,0.589407,1.000000,-0.047312,0.163732
calculated_host_listings_count,0.019517,-0.114713,0.057472,0.127960,-0.072376,-0.047312,1.000000,0.225701
availability_365,-0.010983,0.082731,0.081829,0.144303,0.172028,0.163732,0.225701,1.000000


###  turn the price variable from numeric into binary. 

In [42]:
df_full_train["above_average"] = (df_full_train.price >= 152).astype(int)

In [43]:
df_full_train[:3].T

,0,1,2
neighbourhood_group,brooklyn,manhattan,brooklyn
latitude,40.71577,40.84917,40.68993
longitude,-73.9553,-73.94048,-73.95947
room_type,entire_home/apt,private_room,private_room
price,295,70,58
minimum_nights,3,2,2
number_of_reviews,11,2,0
reviews_per_month,0.87,0.16,0.0
calculated_host_listings_count,1,1,2
availability_365,1,0,0


# Question #3

In [44]:
from sklearn.metrics import mutual_info_score

### **Mutual information** - concept from information theory, it tells us how much we can learn about one variable if we know the value of another

1. Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.

In [45]:
categorical_cols

['neighbourhood_group', 'room_type']

In [46]:
round(mutual_info_score(df_full_train["above_average"], df_full_train["neighbourhood_group"]), 2)

0.05

In [47]:
round(mutual_info_score(df_full_train["above_average"], df_full_train["room_type"]), 2)

0.14

# Question #4

### One-Hot Encoding

In [48]:
from sklearn.feature_extraction import DictVectorizer

In [49]:
DictVectorizer?

In [50]:
dv = DictVectorizer(sparse=False)

In [51]:
categorical_cols

['neighbourhood_group', 'room_type']

In [52]:
cat_num_cols = list(df_train.columns)

In [53]:
df_train.nunique()

neighbourhood_group                   5
latitude                          15246
longitude                         12052
room_type                             3
minimum_nights                       93
number_of_reviews                   354
reviews_per_month                   860
calculated_host_listings_count       47
availability_365                    366
dtype: int64

In [54]:
train_dict = df_train[cat_num_cols].to_dict(orient="records")
train_dict[7]

{'neighbourhood_group': 'manhattan',
 'latitude': 40.82734,
 'longitude': -73.94718,
 'room_type': 'entire_home/apt',
 'minimum_nights': 4,
 'number_of_reviews': 3,
 'reviews_per_month': 1.13,
 'calculated_host_listings_count': 1,
 'availability_365': 6}

In [55]:
X_train = dv.fit_transform(train_dict)
X_train[3]

array([  0.     ,   1.     ,  40.66448, -73.99407,   2.     ,   0.     ,
         1.     ,   0.     ,   0.     ,   0.     ,   3.     ,   0.08   ,
         1.     ,   0.     ,   0.     ])

In [56]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=bronx',
 'neighbourhood_group=brooklyn',
 'neighbourhood_group=manhattan',
 'neighbourhood_group=queens',
 'neighbourhood_group=staten_island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=entire_home/apt',
 'room_type=private_room',
 'room_type=shared_room']

In [57]:
val_dict = df_val[cat_num_cols].to_dict(orient='records')
X_val = dv.transform(val_dict)

###  train a logistic regression

In [58]:
from sklearn.linear_model import LogisticRegression

In [59]:
# LogisticRegression?

In [60]:
y_train[10:15]

10     69
11    105
12    109
13     49
14    160
Name: price, dtype: int64

In [61]:
y_btrain = (y_train >= 152).astype(int)

In [62]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
# solver='lbfgs' is the default solver in newer version of sklearn
# for older versions, you need to specify it explicitly
model.fit(X_train, y_btrain)

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [63]:
dicts_val = df_val[cat_num_cols].to_dict(orient="records")

In [64]:
X_val = dv.transform(dicts_val)

In [65]:
y_pred = model.predict_proba(X_val)[:,1]
y_pred

array([0.03267929, 0.5763682 , 0.41073136, ..., 0.09373391, 0.03354893,
       0.63364242])

In [66]:
pred = (y_pred > 0.5).astype(int)
pred

array([0, 1, 0, ..., 0, 0, 1])

In [67]:
y_bval =  (y_val >= 152).astype(int)
y_bval

0       0
1       0
2       1
3       0
4       1
       ..
9774    1
9775    0
9776    0
9777    0
9778    0
Name: price, Length: 9779, dtype: int32

In [68]:
y_val[10:15]

10    185
11     90
12    125
13    560
14    104
Name: price, dtype: int64

In [69]:
dicts_test = df_test[cat_num_cols].to_dict(orient="records")

In [70]:
X_test = dv.transform(dicts_test)

In [71]:
y_pred = model.predict_proba(X_test)[:,1]
y_pred

array([0.44785728, 0.01401714, 0.08797759, ..., 0.01376209, 0.63738892,
       0.21241861])

In [72]:
def accuracy(og, pred):
    pred = (pred > 0.5).astype(int)
    y_b = (og >= 152).astype(int)
    accuracy = round((pred == y_b).mean(), 4)
    return accuracy

In [73]:
accuracy(y_val, y_pred)

0.5836

In [74]:
def train(train, y_train, cols):
    dv = DictVectorizer(sparse=False)
    train_dict = df_train[cols].to_dict(orient="records")
    X_train = dv.fit_transform(train_dict)

    # training 
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    y_btrain = (y_train >= 152).astype(int)
    model.fit(X_train, y_btrain)
    return model, dv
    
#     # prediction
#     dicts = df[cols].to_dict(orient="records")
#     X_pred = dv.transform(dicts)
#     y_pred = model.predict_proba(X_val)[:,1]
    

In [75]:
model, dv = train(df_train, y_train, cat_num_cols)

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [76]:
def predict(model,dv, df, y, cols):
    dicts = df[cols].to_dict(orient="records")
    X_pred = dv.transform(dicts)
    y_pred = model.predict_proba(X_pred)[:,1]

    return accuracy(y, y_pred)

In [77]:
predict(model,dv, df_val, y_val, cat_num_cols)

0.7863

In [78]:
def train_predict(df_train, y_train, df_check, y_check, cols):
    model, dv = train(df_train, y_train, cols)
 
    accuracy = predict(model, dv,  df_check, y_check, cols)
    return accuracy

In [79]:
og_acc_val = train_predict(df_train, y_train, df_val, y_val, cat_num_cols)
og_acc_val

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7863

#### TEST

In [80]:
og_acc_test = train_predict(df_train, y_train, df_test, y_test, cat_num_cols)
og_acc_test

E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7933

# Question #5

## Feature Elimination

In [81]:
cols = cat_num_cols.copy()

#### Removing neighbourhood_group

In [82]:
cols = cat_num_cols.copy()
cols.remove("neighbourhood_group")
print("Columns used: \n", cols)

val_acc = train_predict(df_train, y_train, df_val, y_val, cols)

test_acc = train_predict(df_train, y_train, df_test, y_test, cols)

print(f"\n\nvalidation set accuracy: {val_acc} \ntest set accuracy: {test_acc}")

Columns used: 
 ['latitude', 'longitude', 'room_type', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


validation set accuracy: 0.751 
test set accuracy: 0.7528


In [83]:
print(f"test diff: {round(og_acc_test - test_acc,4)} \nval diff: {round(og_acc_val - val_acc, 4)}")

test diff: 0.0405 
val diff: 0.0353


#### Removing room_type

In [84]:
cols = cat_num_cols.copy()
cols.remove("room_type")
print("Columns used: \n", cols)

val_acc = train_predict(df_train, y_train, df_val, y_val, cols)

test_acc = train_predict(df_train, y_train, df_test, y_test, cols)

print(f"\n\nvalidation set accuracy: {val_acc} \ntest set accuracy: {test_acc}")

Columns used: 
 ['neighbourhood_group', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(




validation set accuracy: 0.7163 
test set accuracy: 0.7274


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [85]:
print(f"test diff: {round(og_acc_test - test_acc,4)} \nval diff: {round(og_acc_val - val_acc, 4)}")

test diff: 0.0659 
val diff: 0.07


#### Removing number_of_reviews

In [86]:
cols = cat_num_cols.copy()
cols.remove("number_of_reviews")
print("Columns used: \n", cols)

val_acc = train_predict(df_train, y_train, df_val, y_val, cols)

test_acc = train_predict(df_train, y_train, df_test, y_test, cols)

print(f"\n\nvalidation set accuracy: {val_acc} \ntest set accuracy: {test_acc}")

Columns used: 
 ['neighbourhood_group', 'latitude', 'longitude', 'room_type', 'minimum_nights', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(




validation set accuracy: 0.7869 
test set accuracy: 0.7935


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [87]:
print(f"test diff: {round(og_acc_test - test_acc,4)} \nval diff: {round(og_acc_val - val_acc, 4)}")

test diff: -0.0002 
val diff: -0.0006


#### Removing reviews_per_month

In [88]:
cols = cat_num_cols.copy()
cols.remove("reviews_per_month")
print("Columns used: \n", cols)

val_acc = train_predict(df_train, y_train, df_val, y_val, cols)

test_acc = train_predict(df_train, y_train, df_test, y_test, cols)

print(f"\n\nvalidation set accuracy: {val_acc} \ntest set accuracy: {test_acc}")

Columns used: 
 ['neighbourhood_group', 'latitude', 'longitude', 'room_type', 'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count', 'availability_365']


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(




validation set accuracy: 0.7858 
test set accuracy: 0.7918


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [89]:
print(f"test diff: {round(og_acc_test - test_acc,4)} \nval diff: {round(og_acc_val - val_acc, 4)}")

test diff: 0.0015 
val diff: 0.0005


# Question #6

In [90]:
from sklearn.linear_model import Ridge

#### One-Hot encoding

In [112]:
dv = DictVectorizer(sparse=False)
train_dict = df_train[cat_num_cols].to_dict(orient="records")
X_train = dv.fit_transform(train_dict)

In [113]:
model = Ridge(alpha=0.01)
model.fit(X_train, np.log1p(y_train))

Ridge(alpha=0.01)

In [114]:
val_dict = df_val[cat_num_cols].to_dict(orient="records")
X_val = dv.transform(val_dict)

In [118]:
y_pred = model.predict(X_val)
y_pred

array([4.11405448, 5.21395699, 5.01012603, ..., 4.5595196 , 4.19404933,
       5.14435076])

In [116]:
def rmse(y, y_pred):
  se = (y - y_pred) ** 2
  mse = se.mean()
  return np.sqrt(mse)

In [117]:
rmse(np.log1p(y_val), y_pred)

0.49711730461906295

In [125]:
for i in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=i)
    model.fit(X_train, np.log1p(y_train))

    val_dict = df_val[cat_num_cols].to_dict(orient="records")
    X_val = dv.transform(val_dict)
    
    y_pred = model.predict(X_val)
    score = round(rmse(np.log1p(y_val), y_pred),3)
    
    print(f"for alpha={i} rmse={score}")

for alpha=0 rmse=0.497
for alpha=0.01 rmse=0.497
for alpha=0.1 rmse=0.497
for alpha=1 rmse=0.497
for alpha=10 rmse=0.498
